In [33]:
from nltk.tokenize import word_tokenize
from collections import Counter
import collections
import sys
import MeCab
import re
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn import preprocessing
from sklearn.svm import SVC, LinearSVC
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import pandas as pd

from sklearn import cross_validation
from sklearn import datasets
from sklearn import svm
from sklearn.cross_validation import KFold

C:\Users\fmv\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [54]:
#名詞限定
mecab = MeCab.Tagger()
def tokenize(text):
    with open(text, 'r')as f:
        textdata=f.read()#一括で詠み込む  
        textdata = re.split('-{5,}', textdata)[0]+ re.split('-{5,}', textdata)[2]
        textdata = re.split('底本', textdata)[0]
        #詠み込んだ文章を底本部分で分割し本文のみを取得
        textdata = re.split('［＃ここから', textdata)[0]
        #詠み込んだ文章を［＃ここから　部分で分割し本文のみを取得
        lines = textdata.split("\r\n")#改行で分ける
        
        results = []
        for line in lines:
            # 学習に使わない表現の削除処理
            s = line
            s = re.sub('｜', '',s)
            s = re.sub('《.+?》', '', s)# 《》を消す
            s = re.sub('［.+?］', '', s)#[]
            s = re.sub('（.+?）', '', s)#()

            token_list=[]
            node = mecab.parseToNode(s)#parseToNode()を使うと形態素の詳細情報
    
            while node:#ループにして情報を取得していく
                # 語を取得
                word = node.surface#surfaceで表層形
                # 品詞を取得
                part = node.feature.split(",")[0]#featureで形態素情報　[0]で品詞取得

                if part in ["名詞"]:#取得した品詞が一致するなら
                     token_list.append(word)#リストに追加   
                node = node.next
            rl = (" ".join(token_list)).strip()
            results.append(rl)
        w_file = "result_"+text
        with open(w_file, 'w', encoding='utf-8') as wf:
            wf.write("\n".join(results))  


tokenize('yukionna.txt') 
tokenize('noctilucae.txt') 
tokenize('hashino_uede.txt') 
tokenize('a_dead_secret.txt') 
tokenize('at_a_railway_station.txt') 

tokenize('ichiya.txt') 
tokenize('yukionna.txt') 
tokenize('carlyle_hakubutsukan.txt') 
tokenize('gubijinso.txt') 
tokenize('ganjitsu.txt') 
#, "ignore"

In [24]:
t1='result_yukionna.txt'
t2='result_noctilucae.txt'
t3='result_hashino_uede.txt'
t4='result_a_dead_secret.txt'
t5='result_at_a_railway_station.txt'

vectorizer = CountVectorizer(input='filename')
corpus=[t1,t2,t3,t4,t5]  
matrix = vectorizer.fit_transform(corpus)
#CountVectorizer で得られた単語文書行列を，TfidfTransformer でtf-idfの行列に変換する
tfidf = TfidfTransformer()
tfidf_matrix = tfidf.fit_transform(matrix)
np.set_printoptions(precision=2) 


#numpy の配列 ndarray のソーティングより

#vectorizer.get_feature_names() 特徴名（単語）
#tfidf_matrix.toarray()  # tf-idfの行列
name_list = np.array(vectorizer.get_feature_names())
population = np.array(tfidf_matrix.toarray())
print(population)
indices = np.argsort(-population)
#品詞を名詞に限定した場合
#大きい値ほど各文書を特徴付ける単語　 値の多い順に20件表示
print(name_list[indices[::,0:20]])

[[ 0.    0.    0.   ...,  0.    0.    0.  ]
 [ 0.    0.    0.   ...,  0.06  0.12  0.12]
 [ 0.    0.04  0.   ...,  0.    0.    0.  ]
 [ 0.    0.    0.04 ...,  0.    0.    0.  ]
 [ 0.04  0.    0.   ...,  0.    0.    0.  ]]
[['彼女' '巳之吉' 'あなた' '茂作' '小屋' 'よう' '綺麗' '子供' 'それ' '老人' '渡し守' 'わし' 'お前' '彼等'
  '吹雪' '少女' 'あと' '江戸' '自分' '約束']
 ['生命' 'よう' '夜光虫' '思惟' '無限' '変化' '存在' '恒星' 'もの' '波動' '輝き' '深紅' '一瞬' '天の河'
  '小浪' '翠玉' '水平' '閃光' '黄色' '鼓動']
 ['たち' 'わし' '平七' '士官' '欄干' 'よう' '熊本' 'とき' '薩摩' '遠く' 'らい' 'そこ' 'ここ' 'ところ'
  '農家' 'みんな' '薩軍' '辺り' '灰色' '鎮台']
 ['和尚' '抽斗' '箪笥' '部屋' '貴女' 'もの' 'それ' '手紙' '心配' '貼り紙' 'よう' 'つぎ' '飾り' '修業'
  '住職' '秘密' '一同' '道具' '教育' '一番']
 ['犯人' 'たち' 'こと' '子ども' '警部' '坊や' '警察' '熊本' 'とき' 'よう' 'あんた' '警官' '巡査' 'ここ'
  '未亡人' '改札' '殺人' '犯罪' '護送' '兇徒']]


In [3]:
t1='result_ichiya.txt'
t2='result_yukionna.txt'
t3='result_carlyle_hakubutsukan.txt'
t4='result_gubijinso.txt'
t5='result_ganjitsu.txt'

vectorizer = CountVectorizer(input='filename')
corpus=[t1,t2,t3,t4,t5]  
matrix = vectorizer.fit_transform(corpus)
#CountVectorizer で得られた単語文書行列を，TfidfTransformer でtf-idfの行列に変換する
tfidf = TfidfTransformer()
tfidf_matrix = tfidf.fit_transform(matrix)
np.set_printoptions(precision=2) 

#numpy の配列 ndarray のソーティングより

#vectorizer.get_feature_names() 特徴名（単語）
#tfidf_matrix.toarray()  # tf-idfの行列
name_list = np.array(vectorizer.get_feature_names())
population = np.array(tfidf_matrix.toarray())
#print(population)
indices = np.argsort(-population)
#品詞を名詞に限定した場合
#大きい値ほど各文書を特徴付ける単語　 値の多い順に20件表示
print(name_list[indices[::,0:20]])

[['蜘蛛' '脚気' '蚊遣' '多く' '一疋' '三つ' 'ククー' '彼ら' '菓子' '葛餅' '廻廊' '膝頭' '美人' 'くし'
  '床柱' '合奏' 'もの' 'あたり' '一刻' '座敷']
 ['彼女' '巳之吉' '茂作' 'あなた' '小屋' '子供' 'よう' '綺麗' '老人' 'それ' '渡し守' 'お前' '少年' '彼等'
  '約束' '少女' '江戸' '吹雪' '自分' 'わし']
 ['カーライル' '婆さん' '倫敦' 'もの' 'チェルシー' 'よう' '演説' 'そう' '案内' 'これ' '細君' '四角' '部屋'
  'わし' 'ここ' '往来' '村夫子' '御前' 'チェイン' '夫人']
 ['甲野' 'さん' 'もの' '反吐' '雅号' '叡山' 'どこ' 'よう' '静か' 'おれ' '四角' 'だい' 'ハハハハ' '愛嬌'
  '哲学' '余計' '馬鹿' 'いや' '大原女' '袖無']
 ['元日' '新聞' '告白' '想像' 'もの' '一昨年' '具眼' '年始' '何処' '屑屋' '上手' '虚子' '困難' '読者'
  '正月' '去年' '世間' 'それ' '調子' '材料']]


In [20]:
doc_info = pd.read_csv('token_files.csv',encoding='cp932')
print(doc_info)
files = doc_info['文書'].values
print(files)
author_codes = doc_info['著者コード'].values 
print(author_codes)

[0 0 0 0 0 1 1 1 1 1]


In [28]:
docs_train    = population[:6]        # 訓練データ
authors_train = author_codes[:6]  # 正解ラベル
docs_test     = population[6:]         # テストデータ
authors_test  = author_codes[6:]   # 正解ラベル

In [34]:
X =np.array(population)
y =np.array(author_codes)

X_train, X_test, y_train, y_test = cross_validation.train_test_split(X, y, test_size=0.25, random_state=0)
kf = KFold(5, n_folds=3)
for train, test in kf:
    X_train, y_train, X_test, y_test = X[train], y[train], X[test], y[test]
    clf = svm.SVC(kernel='linear', C=1).fit(X_train, y_train)
    score = clf.score(X_test, y_test)
    print(score)


ValueError: Found input variables with inconsistent numbers of samples: [5, 10]

In [ ]:
clf = SVC(kernel='linear', random_state=None)
clf.fit(docs_train ,authors_train)
output = clf.predict(docs_test)
print('分類結果:', output)

In [ ]:
doc_info.to_csv('token_files.csv', columns=['文書'])
#著者コード

#t1='result_akai_heya.txt'
#t2='result_carmen.txt'
#t3='result_hayasugiru_maiso.txt'
#t4='result_meijinden.txt'
#t1='result_akai_heya.txt'
#t2='result_carmen.txt'
#t3='result_hayasugiru_maiso.txt'
#t4='result_meijinden.txt'


#t2='result_carmen.txt'

with open(r'C:\Users\fmv\Desktop\python\12\result_carmen.txt', 'r',encoding="utf-8_sig")as f:
    t2=f.read()#一括で詠み込む
